In [38]:
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
from os import listdir
import re
ls =listdir('./20_newsgroups')
dict_features = {}
for folder in ls:
    files=listdir('./20_newsgroups/'+folder)
    for file in files:
        f=open('./20_newsgroups/'+folder+'/'+file,'r',encoding="ISO-8859-1")
        try:
            text = f.read()
            tokens = re.compile('\w+').findall(text)
            words = [token.lower() for token in tokens if ( token.isalpha() and len(token)>1 and token not in stop_words)]
            for word in words:
                if word in dict_features:
                    dict_features[word]+=1
                else:
                    dict_features[word]=1
        except:
            print('Error in reading file')

Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file


In [23]:
import operator
sorted_values = sorted(dict_features.items(), key=operator.itemgetter(1),reverse = True)
sorted_dict = dict(sorted_values)
features = sorted_dict.keys()
selected_features=[]
for feature in features:
    if feature not in stop_words:
        selected_features.append(feature)
selected_features=selected_features[0:1000]

In [24]:
X_data=[]
Y_data=[]
for folder in ls:
    files=listdir('./20_newsgroups/'+folder)
    for file in files:
        f=open('./20_newsgroups/'+folder+'/'+file,'r',encoding="ISO-8859-1")
        try:
            text = f.read()
            #tokens = word_tokenize(text)
            tokens = re.compile('\w+').findall(text)
            words = [token.lower() for token in tokens if ( token.isalpha() and token not in stop_words and len(token)>1)]
            x=[]
            for i in range(0,1000):
                x.append(0)
            Y_data.append(ls.index(folder))
            for word in words:
                if word in selected_features:
                    pos = selected_features.index(word)
                    x[pos]+=1
            X_data.append(x)
        except:
            print('Error in reading file')


Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file
Error in reading file


In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_data,Y_data)

In [27]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)

In [28]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))

0.798238590873


In [33]:
import numpy as np
def fit(x_train,y_train):
    dict_file={}
    y_classes=set(y_train)
    feature_class=x_train.shape[1]
    dict_file['total']=len(y_train)
    for y in y_classes:
        dict_file[y]={}
        dict_file[y]['total_y']=len(y_train[(y_train==y)])
        x_features=x_train[y_train==y]
        for i in range(1,feature_class+1):
            dict_file[y][i]={}
            possible_values=set(x_train[:,i-1])
            for x in possible_values:
                dict_file[y][i][x]=(x_features[:,i-1]==x).sum()     
    return dict_file

In [34]:
def getPrediction(x,dict):
    y_classes=dict.keys()
    class_predicted=-1
    max_prob=-1000
    for y in y_classes:
        if y != 'total':
            prob=np.log(dict[y]['total_y'])-np.log(dict['total'])
            x_features=dict[y].keys()
            i=0
            for xf in x_features:
                   if xf != 'total_y':
                         if x[i] in dict[y][xf].keys():
                           prob=prob+np.log(dict[y][xf][x[i]]+1)-np.log(dict[y]['total_y']+len(dict[y][xf]))
                         i=i+1
            if prob > max_prob:
                max_prob=prob
                class_predicted=y
    return class_predicted

In [35]:
def predict(x_test,dict_file):
    y_predict=[]
    for x in x_test:
        y_predict.append(getPrediction(x,dict_file))
    return y_predict

In [32]:
dictme=fit(np.array(x_train),np.array(y_train))

In [36]:
y_predict=predict(x_test,dictme)

In [37]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))

0.927942353883


In [22]:
print(selected_features)

['line', 'subject', 'newsgroup', 'date', 'organ', 'path', 'apr', 'gmt', 'would', 'write', 'one', 'use', 'refer', 'articl', 'like', 'news', 'get', 'sender', 'univers', 'peopl', 'know', 'think', 'say', 'time', 'system', 'thi', 'make', 'also', 'go', 'may', 'work', 'could', 'new', 'want', 'good', 'right', 'year', 'xref', 'see', 'need', 'even', 'way', 'thing', 'look', 'well', 'problem', 'christian', 'usenet', 'god', 'state', 'uunet', 'distribut', 'mani', 'much', 'tri', 'file', 'first', 'us', 'two', 'comput', 'question', 'max', 'take', 'world', 'call', 'believ', 'window', 'post', 'come', 'point', 'program', 'mean', 'anyon', 'said', 'ca', 'rochest', 'seem', 'run', 'help', 'read', 'differ', 'pleas', 'realli', 'someth', 'find', 'drive', 'number', 'gatech', 'includ', 'sinc', 'back', 'inform', 'thank', 'govern', 'day', 'reason', 'still', 'game', 'give', 'person', 'start', 'case', 'gener', 'support', 'part', 'udel', 'law', 'follow', 'might', 'ask', 'better', 'last', 'sure', 'fact', 'group', 'never